# Préambule

Ce notebook correspond à la base de données Neo4j réalisée dans le cadre du master EBAM 2019 - 2020 par Anthony Moisan et Léonard Péan

Prérequis, il est nécessaire :
* de lancer en local un serveur Neo4j 
* de mettre les fichiers csv dans le fichier d'import de la base de données via Manage -> Open Folders
* d'installer le plugin APOC via Manage -> Plugins -> APOC

# Small Example

Petit jeu de données créés à la main pour voir la pertinence des requêtes. Niveau1_Small et Niveau2_Small

## Packages

Installation des packages nécessaires à l'execution

In [ ]:
!pip install py2neo
import pandas as pd

## Connexion à la base de données

In [ ]:
# les trois premiers éléments sont à configurer en fonction de la création en local de la base de données Neo4j
from py2neo import Graph
myUri = "http://localhost:7474"
myUser = "neo4j"
myPassword = "mmas3783"
graph = Graph(myUri, username=myUser, password=myPassword)

## Ensemble des procédures pour créer le graphe ou le détruire

In [ ]:
#Permet de mettre les contraintes d'intégrité et index
def DefineConstraintAndIndex(db):
    queryInsertConstraintEntityCodeLEI = "CREATE CONSTRAINT ON (e:Entity) ASSERT e.code_LEI IS UNIQUE;"
    db.run(queryInsertConstraintEntityCodeLEI)
    queryInsertIndexCountryName = "CREATE INDEX ON :Country(name);"
    db.run(queryInsertIndexCountryName)
    queryInsertIndexEntiteName = "CREATE INDEX ON :Entity(name);"
    db.run(queryInsertIndexEntiteName)
    queryDropIndexActiveName = "CREATE INDEX ON :Active(name);"
    db.run(queryDropIndexActiveName)


In [ ]:
#Permet de définir l'import de Niveau 1
def DefineImportNiveau1(db,namefile):
    queryLevel1 ="USING PERIODIC COMMIT 500 " \
    "LOAD CSV WITH HEADERS FROM 'file:///"+namefile+".csv' " \
    "AS ligne FIELDTERMINATOR ';' " \
    "MERGE (e:Entity {code_LEI: ligne.Code_LEI}) " \
    "SET e.name = ligne.Legal_Name " \
    "SET e.last_update = apoc.date.format(apoc.date.parse(ligne.Last_Update_Date, 'ms', 'dd/MM/yyyy')) " \
    "MERGE (act:Active {name:ligne.Status_Actif}) " \
    "MERGE (country:Country {name: ligne.Country_Code}) " \
    "MERGE (e)-[:ACTIVE]-(act) " \
    "MERGE (e)-[:REGISTERED]-(country)" 
    db.run(queryLevel1)

In [ ]:
#Permet de définir l'import de Niveau 2
def DefineImportNiveau2(db,namefile):
    queryLevel2 ="USING PERIODIC COMMIT 500 " \
    "LOAD CSV WITH HEADERS FROM 'file:///"+namefile+".csv' " \
    "AS ligne FIELDTERMINATOR ';' " \
    "MATCH (startNode:Entity{ code_LEI: ligne.Start_Node}),(endNode:Entity { code_LEI: ligne.End_Node}) " \
    "MERGE (startNode)-[:RELATIONSHIP { role: ligne.Relation_ShipeType }]->(endNode)"
    db.run(queryLevel2)

In [ ]:
#1) On créer les contraintes et index
DefineConstraintAndIndex(graph)
#2) On réalise l'import de niveau 1 pour initialiser une partie du graphe
DefineImportNiveau1(graph,'Niveau1_Small')
#3) On réalise l'import de niveau 2 pour mettre les relations entre les entités juridiques
DefineImportNiveau2(graph,'Niveau2_Small')

In [ ]:
#Permet de supprimer l'ensemble des noeuds et des relations 
def Armageddon(db):
    queryDeleteNodesAndRelations = "MATCH (n) OPTIONAL MATCH (n)-[r]-() DELETE n,r"
    db.run(queryDeleteNodesAndRelations)    

In [ ]:
#Permet de supprimer les contraintes d'intégrité et les index
def DropConstraintAndIndex(db):
    queryDropIndexCountryName = "DROP INDEX ON :Country(name);"
    db.run(queryDropIndexCountryName)
    queryDropIndexEntiteName = "DROP INDEX ON :Entity(name);"
    db.run(queryDropIndexEntiteName)
    queryDropIndexActiveName = "DROP INDEX ON :Active(name);"
    db.run(queryDropIndexActiveName)
    queryDropConstraintEntityCodeLEI = "DROP CONSTRAINT ON (e:Entity) ASSERT e.code_LEI IS UNIQUE;"
    db.run(queryDropConstraintEntityCodeLEI)

## Ensemble des requêtes

### Nombre de noeuds actifs

In [ ]:
def VisualizeNodes(db) :
    qCount = "MATCH (e:Entity)-[:ACTIVE]-(act) " \
            "WHERE act.name = 'ACTIVE' " \
            "RETURN count(e) "
    numberOfActiveNodes = db.run(qCount).evaluate()
    qCountTotal = "MATCH (e:Entity)-[:ACTIVE]-(act) " \
                "RETURN count(e) "
    numberOfNodes = db.run(qCountTotal).evaluate()
    print("Number of active nodes  : " + str(numberOfActiveNodes))
    print("Number of total nodes  : " + str(numberOfNodes))
    print("% of active nodes : " + str(round(numberOfActiveNodes/numberOfNodes*100,2)))

In [ ]:
VisualizeNodes(graph)

### Nombre d'entités légales par pays par ordre croissant

In [ ]:
def EntityPerCountry(db) :
    query = "MATCH (c:Country)-[:REGISTERED]-(e:Entity) " \
    "RETURN distinct(c.name) as Country, count(e) as Number ORDER BY Number DESC"
    return pd.DataFrame(db.run(query).data())

In [ ]:
EntityPerCountry(graph)

Cela correspond aux nationalités des filiales fournies dans le cas jouet

### Schema consolidation AXA

#### 1ère requête sans prise en compte du caractère active ou non de la filiale

On cherche à partir d'une société à avoir toutes les filiales qui en dépendent

In [ ]:
def SchemaConsolidationAxa(db,society) :
    query = "MATCH (e:Entity {name:"+society+"}), " \
    "path = (e)-[:RELATIONSHIP*]-(subsdiary:Entity) " \
    "return extract(x IN nodes(path) | x.name) as Nodes,length(path) as Depth ORDER BY Depth ASC"
    return pd.DataFrame(db.run(query).data())


In [ ]:
SchemaConsolidationAxa(graph, "'AXA'")

Schéma cohérent avec Axa en prenant en compte dans le cas présent les filiales inactives

#### 2ème requête en prenant le caractère active des filiales

In [ ]:
def SchemaConsolidationWithActiveSubsidiary(db, society) :
    query = "MATCH (e:Entity {name:" + society + "}), " \
    "path = (e)-[:RELATIONSHIP*]-(subsidiary:Entity)-[:ACTIVE]-(act {name:'ACTIVE'}) " \
    "RETURN extract(x IN nodes(path) WHERE x.name <> 'ACTIVE' | x.name) as Nodes,length(path) as Depth ORDER BY Depth ASC"
    return pd.DataFrame(db.run(query).data())

In [ ]:
SchemaConsolidationWithActiveSubsidiary(graph, "'AXA'")

Schéma cohérent avec les filiales actives d'Axa

### Calcul de profondeur entre société mère et filiales

In [ ]:
#Permet de savoir si une société est la société mère (elle n'a pas de relations où elle est le noeud de sortie dans une relation de filiation avec d'autres entités juridiques)
#On ajoute un champ au noeud entité IsParent
def DefineHolding(db) :
    queryIsParentFalse = "MATCH (e:Entity)<-[RELATIONSHIP]-() " \
    "SET e.isParent = FALSE " \
    "RETURN e"
    db.run(queryIsParentFalse)
    
    queryIsParentTrue = "MATCH (e:Entity) " \
    "WHERE not exists(e.isParent) " \
    "SET e.isParent = TRUE " \
    "return e"
    db.run(queryIsParentTrue)

In [ ]:
DefineHolding(graph)

In [ ]:
#Permet de calculer la profondeur entre les sociétés mères d'un pays et ses filiales
def CalculateHoldingSubsidiaryPerCountry(db, nameCountry):
    query = "MATCH (c:Country {name:"+nameCountry+"})-[:REGISTERED]-(e:Entity {isParent:TRUE})-[:ACTIVE]-(act{name:'ACTIVE'}), " \
    "path = (e)-[:RELATIONSHIP*]-(subsidiary:Entity)-[:ACTIVE]-(act{name:'ACTIVE'}) " \
    "return e.name as Holding, subsidiary.name as Subsidiary, length(path) as Depth ORDER BY Depth DESC "
    return pd.DataFrame(db.run(query).data())

In [ ]:
CalculateHoldingSubsidiaryPerCountry(graph, "'FR'")

Les résultats correspondent aux filiales uniquement actives d'Axa et de SMA

### Identification de l'évasion fiscale potentielle

In [ ]:
#Permet de voir les sociétés mères d'un pays qui ont des filiales actives dans des paradis fiscaux (listCountrySuspect)
def IdentifySuspectSubsidiaries(db,nameCountry, listCountrySuspect):
    query = "MATCH (c:Country {name:"+nameCountry+"})-[:REGISTERED]-(e:Entity {isParent:TRUE})-[:ACTIVE]-(act{name:'ACTIVE'}), " \
    "path = (e)-[:RELATIONSHIP*]-(subsidiary:Entity)-[:REGISTERED]-(cSubsidiary:Country) WHERE cSubsidiary.name in "+listCountrySuspect+ "AND (subsidiary:Entity)-[:ACTIVE]-(act{name:'ACTIVE'})" \
    "return e.name as Entity, subsidiary.name as Subsidiary, length(path) as Depth"
    return pd.DataFrame(db.run(query).data())

In [ ]:
listCountrySuspect = "['LU', 'BS']" #permet d'identifier le luxembourg et Les iles Bahamas
IdentifySuspectSubsidiaries(graph,"'FR'", listCountrySuspect )

Le résultat ne fait pas apparaître la filiale non active dans les Bahamas associée à la société SMA 

## Destruction du graph

In [ ]:
Armageddon(graph)
DropConstraintAndIndex(graph)

# Jeu de données réelles
Les jeux de données réelles sont basées sur la transformation XML des données accessibles en Open Data sur le site https://www.gleif.org via un CSV. Le code Python permet de faire la conversion se trouve aussi dans un autre projet Python dans SRC

## Creation du graph

In [ ]:
#1) On créer les contraintes et index
DefineConstraintAndIndex(graph)
#2) On réalise l'import de niveau 1 pour initialiser une partie du graphe
DefineImportNiveau1(graph,'Niveau1_Real')
#3) On réalise l'import de niveau 2 pour mettre les relations entre les entités juridiques
DefineImportNiveau2(graph,'Niveau2_Real')

### Nombre de noeuds actifs

In [ ]:
VisualizeNodes(graph)

### Nombre d'entités légales par pays par ordre croissant

In [ ]:
EntityPerCountry(graph)

### Schema consolidation Société X

In [ ]:
SchemaConsolidationWithActiveSubsidiary(graph, "'X'")

### Calcul de profondeur entre société mère et filiales

In [ ]:
DefineHolding(graph)
CalculateHoldingSubsidiaryPerCountry(graph, "'FR'")

### Identification de l'évasion fiscale potentielle

In [ ]:
listCountrySuspect = "['LU', 'BS']" #permet d'identifier le luxembourg et Les iles Bahamas
IdentifySuspectSubsidiaries(graph,"'FR'", listCountrySuspect )

## Destruction du graph

In [ ]:
Armageddon(graph)
DropConstraintAndIndex(graph)

# Conclusion

Dans le cadre de ce projet, nous avons pu :
* manipuler une base de données Neo4j de sa création, avec son langage spécifique de requêtes, jusqu'à sa suppression
* voir la force d'une base de données relationnelles pour identifier les relations d'amis / d'amis..., qui est un vrai point fort de Neo4j par rapport au bdd relationnelle
* améliorer nos compétences en Python pour réaliser les transformations de données et appeler directement Neo4j/Cypher via py2neo. 

Les difficultés auxquelles nous avons été confrontées :
* la difficulté lorsque Cypher ne renvoie rien pour identifier l'erreur dans les requêtes
* le coût d'un langage qui n'est pas évident par exemple dans la prise en comptes des dates dans les propriétés et de la librairie py2neo qui s'est avérée plus simple par rapport à l'API Rest
